# Breast Density Classification in MRI: comparing Deep learning with Radiolomics

Acknowledgements:

- https://jacobgil.github.io/pytorch-gradcam-book
- https://github.com/Astarakee/Radiomics_pipeline
- https://github.com/shijianjian/EfficientNet-PyTorch-3D

T1w sequences were used for density estimation in this project, gold standard was BIRADS density assessment from radiologists' visual inspection.


## 1. Four-category  classification

The breast density was difined as:
- 1) almost entirely fatty
- 2) scattered fibroglandular tissue
- 3) heterogeneously dense
- 4) etremely dense

based on the volume of the fibroglandular tissue in the breast.

The most intuitive way is to train a 4-class classifier based on categories.

### 1.1 Deep learning 4-class classification 

In [ ]:
import os
import sys
import glob
import random
import yaml
import io
import time
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import torch
from torch import nn
from torch.utils import data as torch_data
from torchvision import transforms as T
from sklearn import model_selection as sk_model_selection
import torch.nn.functional as F
import torchio as tio
from sklearn.metrics import roc_auc_score, roc_curve, cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay,RocCurveDisplay
import joblib
import pickle
import warnings
import xlwt
import scipy.stats

In [ ]:
from data_load import load_croped_volumn, Dataset
from utilities import bootstraps, make_confusion_matrix
from radiomics_cls import test_clf
from dl_cls import predict

#### Define dataset
All information were keeped in a table with three colomuns: patient_id, density_label and Filepath

In [ ]:
############
# path to the 3D volumes
############
data_root = 'xxxx'

#########
# Load the dataset table
#########
test_df = pd.read_excel(r"../DATA_FOR_TEST_NO_PROTHESIS_WITHPATH.xlsx")
test_df.reset_index(drop = True, inplace = True)

#### Load model

Load efficientbet-3D model
original implementation: https://github.com/shijianjian/EfficientNet-PyTorch-3D

In [ ]:
pytorch3dpath = "EfficientNet-PyTorch-3D"
sys.path.append(pytorch3dpath)
from efficientnet_pytorch_3d import EfficientNet3D
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Model(nn.Module):
    def __init__(self, net = 'b3', classes = 4 ):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-{}".format(net), override_params={'num_classes': classes},
                                            in_channels=1)
    
    def forward(self, x):
        out = self.net(x)
        return out

#### Predcit function

In [ ]:
checkpoint = 'models/T1W_150420_sgd_four_class_no_prothesis_b3-e44-loss0.757-acc0.714.pth'

## tranformer for test, no data augmentation
valid_transformer = tio.Compose([
        tio.Resize([150, 420, 144], image_interpolation = 'bspline'),
        tio.RescaleIntensity(out_min_max=(0, 1)),
    ])

dl_pred_4 = predict(checkpoint, 'b3', test_df, data_root = data_root, transformer = valid_transformer,
               binary = False, dense = False, targets = True, gradcam = False)

In [ ]:
dl_pred_4.tail()

In [ ]:
########
# save the predicted probobilities
#dl_pred_4.to_csv('dl_pred_val_4.csv', index=False)

In [ ]:
y_pred = list(dl_pred_4.Prediction.values)
y_ture = list(dl_pred_4.Label.values)

cm = confusion_matrix(y_ture, y_pred)

categories = ['Fatty', 'Scattered', 'Heterogeneous', 'Dense']

make_confusion_matrix(cm,group_names=None,
                          categories=categories,
                          count=True,
                          percent=True,
                          cbar=False,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          figsize=None,
                          cmap='Blues',
                          title=None)

### 1.2 Radiomics 4-class classification

In [ ]:
with open("feature_names.yaml", 'r') as stream:
    feature_names = yaml.safe_load(stream)['feature list']

In [ ]:
############
# Load radiomics features extracted, for feature extraction, please refer to readme
############
path_to_test_radiomics = 'xxxx/radiomic_features_test.csv'

In [ ]:
test_df_radiomics = pd.read_csv(path_to_test_radiomics)
test_feature_df = test_df_radiomics.drop(columns=feature_names)
x_test, y_test = test_feature_df.values, test_label_set
print('the size of the test subset is {}'.format(x_test.shape))

In [ ]:
####
# feature normalization
####
x_test = feature_normalization(x_test)

In [ ]:
############
# Load trained logestic regression, random forest and SVM models
############
clf_lr_4 = joblib.load('models/lr_4.pkl')
clf_rf_4 = joblib.load('models/rf_4.pkl')
clf_svm_4 = joblib.load('models/svm_4.pkl')

In [ ]:
############
# get prediction from trained models with test data
############
_, y_pred_rm_4, _,  _, _, _ = test_clf(clf_rm_4, test_set, test_label)
_, y_pred_svm_4, _,  _, _, _ = test_clf(clf_svm_4, test_set, test_label)
_, y_pred_lr_4, _,  _, _, _ = test_clf(clf_lr_4, test_set, test_label)

## 2 Binary classification

Instead of classifiy the density into 4 categroies, we classificy breast into dense/nondense, in which 1) almost entirely fatty and 2) scattered fibroglandular tissue were defined as nondense, and 3) heterogeneously dense and 4) etremely dense were defined as dense.

### 2.1 Dense/Non dense classification (task (ii))

#### 2.1.1 Radiomics method

In [ ]:
#########
# Transfer 4 category labels to binary label
#########
test_label_binary = [0 if x < 2 else 1 for x in test_label]

In [ ]:
############
# Load trained logestic regression, random forest and SVM models
############
clf_lr_2 = joblib.load('models/lr_2.pkl')
clf_rf_2 = joblib.load('models/rf_2.pkl')
clf_svm_2 = joblib.load('models/svm_2.pkl')

In [ ]:
############
# get prediction from trained models with test data
############
_, y_pred_rf_2,_, _,_,_ = test_clf(clf_rf_2, test_set, test_label_binary)
_, y_pred_lr_2,_, _,_,_ = test_clf(clf_lr_2, test_set, test_label_binary)
_, y_pred_svm_2,_, _,_,_ = test_clf(clf_svm_2, test_set, test_label_binary)

### 2.1.2 Deep laerning method

In [ ]:
#########
# load trained model for binary classification
#########
checkpoint = 'models/T1W_150420_binary_adam_b5-e73-loss0.317-acc0.903.pth'

## tranformer for test, no data augmentation
valid_transformer = tio.Compose([
        tio.Resize([150, 420, 144], image_interpolation = 'bspline'),
        tio.RescaleIntensity(out_min_max=(0, 1)),
    ])

# pred = predict(checkpoint, 'b5', valid_df, data_root = data_root, transformer = valid_transformer,
#                binary = True, dense = False, targets = True, gradcam = False)

dl_pred_2 = predict(checkpoint, 'b5', test_df, data_root = data_root, transformer = valid_transformer,
               binary = True, dense = False, targets = True, gradcam = False)

In [ ]:
y_prob_dl = dl_pred_2.Probility_1

In [ ]:
bootstraps_roc(np.array(y_prob_dl),np.array(test_label_binary))

### 2.1.3 ROC curve

For binary classification, ROC curves could better illustrate the performace of each model. 

In [ ]:
classifiers = {"Linear SVM":clf_svm,
               "Random ForEst":clf_rf,
               "Logistic Regression":clf_lr}

fig, [ax_roc, ax_det] = plt.subplots(1, 2, figsize=(11, 5))

for name, clf in classifiers.items():
    cm, y_pred, y_prob,  fpr, tpr, threshold = test_clf(clf, x_test, y_test)

    RocCurveDisplay.from_predictions(y_test, y_prob[:,1], pos_label = 1, ax=ax_roc, name=name)
    DetCurveDisplay.from_predictions(y_test, y_prob[:,1], pos_label = 1, ax=ax_det, name=name)

RocCurveDisplay.from_predictions(test_label_binary, y_prob_dl, pos_label = 1, ax=ax_roc, name='Deep Learning')
DetCurveDisplay.from_predictions(test_label_binary, y_prob_dl, pos_label = 1, ax=ax_det, name='Deep Learning')

#ax_roc.set_title("Receiver Operating Characteristic (ROC) curves")
#ax_det.set_title("Detection Error Tradeoff (DET) curves")

ax_roc.set_xlabel('False Positive Rate')
ax_roc.set_ylabel('True Positive Rate')

ax_det.set_xlabel('False Positive Rate')
ax_det.set_ylabel('False Negtive Rate')

ax_roc.grid(linestyle="--")
ax_det.grid(linestyle="--")

plt.legend()

### 2.2 Extrem dense/ Non-extrem dense classification task(iii)

#### 2.2.1 Radiomics  extrem / non-extrem classification

In [2]:
##########
# Transform labels, category A,B,C were asigned with 0, category D asigned with label 1
##########
extrem_labels = np.array([1 if x == 3 else 0 for x in label_values])
print(Counter(label_values))

NameError: name 'np' is not defined

In [ ]:
y_prob_dl_exd = dl_pred_4.Probility_3.values

classifiers = {"Linear SVM":clf_svm_4,
               "Random ForEst":clf_rf_4,
               "Logistic Regression":clf_lr_4}

fig, [ax_roc, ax_det] = plt.subplots(1, 2, figsize=(11, 5))

for name, clf in classifiers.items():
    cm, y_pred, y_prob,  fpr, tpr, threshold = test_clf(clf, x_test, extrem_labels)

    RocCurveDisplay.from_predictions(extrem_labels, y_prob[:,3], pos_label = 1, ax=ax_roc, name=name)
    DetCurveDisplay.from_predictions(extrem_labels, y_prob[:,3], pos_label = 1, ax=ax_det, name=name)

RocCurveDisplay.from_predictions(extrem_labels, y_prob_dl_exd, pos_label = 1, ax=ax_roc, name='Deep Learning')
DetCurveDisplay.from_predictions(extrem_labels, y_prob_dl_exd, pos_label = 1, ax=ax_det, name='Deep Learning')

#ax_roc.set_title("Receiver Operating Characteristic (ROC) curves")
#ax_det.set_title("Detection Error Tradeoff (DET) curves")

ax_roc.set_xlabel('False Positive Rate')
ax_roc.set_ylabel('True Positive Rate')

ax_det.set_xlabel('False Positive Rate')
ax_det.set_ylabel('False Negtive Rate')

ax_roc.grid(linestyle="--")
ax_det.grid(linestyle="--")

plt.legend()